In [ ]:
import numpy as np

wind_speed = 12
turbine_efficiency = 0.4
turbine_radius = 50
farm_width = 1000
farm_height = 1000

def objective_function(turbine_positions):
    total_energy = 0
    num_turbines = turbine_positions.shape[0]
    for i in range(num_turbines):
        x1, y1 = turbine_positions[i]
        energy = turbine_efficiency * wind_speed * turbine_radius
        for j in range(i + 1, num_turbines):
            x2, y2 = turbine_positions[j]
            distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            if distance < turbine_radius * 2:
                energy -= (turbine_radius * 2 - distance) * 10
        total_energy += max(0, energy)
    return -total_energy

def cuckoo_search(nests, max_iter, prob_abandon):
    num_turbines = nests[0].shape[0]
    dim = nests[0].shape[1]

    best_solution = np.copy(nests[0])
    best_fitness = objective_function(best_solution)

    for iter in range(max_iter):
        new_nests = []
        for nest in nests:
            step = np.random.normal(0, 1, (num_turbines, dim))
            new_nest = nest + step
            new_nest = np.clip(new_nest, [0, 0], [farm_width, farm_height])
            new_nests.append(new_nest)

        for i in range(len(nests)):
            fitness = objective_function(new_nests[i])
            if fitness < objective_function(nests[i]):
                nests[i] = new_nests[i]
                if fitness < best_fitness:
                    best_solution = np.copy(nests[i])
                    best_fitness = fitness

        worst_nest_indices = np.argsort([objective_function(nest) for nest in nests])[-int(prob_abandon * len(nests)): ]
        for index in worst_nest_indices:
            nests[index] = np.random.uniform(0, farm_width, (num_turbines, dim))

    return best_solution, best_fitness

num_turbines = 5
nests = [np.random.uniform(0, farm_width, (num_turbines, 2)) for _ in range(10)]
max_iter = 100
prob_abandon = 0.3

best_solution, best_fitness = cuckoo_search(nests, max_iter, prob_abandon)
print(f"Best Wind Farm Placement: {best_solution}")
print(f"Best Fitness (Total Energy): {-best_fitness}")


Best Wind Farm Placement: [[733.63320435  95.69672436]
 [597.82866783 914.88505148]
 [958.46347551 734.76656002]
 [302.20328039  38.59805716]
 [838.50931748 978.31225114]]
Best Fitness (Total Energy): 1200.0000000000002
